<a href="https://colab.research.google.com/github/hafsatariq18/LangChain/blob/main/RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

In [ ]:
pip install langchain langchain_community langchain_chroma

In [ ]:
pip install -qU langchain-openai

In [ ]:
pip install pymupdf

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

··········


In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
loader = PyMuPDFLoader("Story.pdf")
documents = loader.load()
print(documents)

[Document(metadata={'source': 'Story.pdf', 'file_path': 'Story.pdf', 'page': 0, 'total_pages': 3, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20240902104121Z', 'modDate': '', 'trapped': ''}, page_content='In a small, remote village surrounded by rolling hills, lived four lifelong friends who had grown up together, each dreaming of a life beyond the \nstruggles they faced. Despite their hardships, they shared an unbreakable bond that had carried them through the toughest of times. These \nfriends were Ali, Kabir, Zoya, and Sara.\n \nAli was the eldest among them and the most responsible. He worked tirelessly as a farmer, tending to his small patch of land with dedication \nand care. Life as a farmer was difficult, with unpredictable weather and meager harvests, but Ali never complained. He was a natural leader, \nalways looking out for his friends and family, and his wisdom guided the grou

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
chunks = text_splitter.split_documents(documents)
print(len(chunks))

24


In [ ]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use two sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "Who was Ali?"})
response["answer"]

'Ali was the eldest among his friends, a responsible farmer who worked tirelessly and showed natural leadership qualities.'

In [ ]:
response = rag_chain.invoke({"input": "What is the story about"})
response["answer"]

'The story is about four friends who embark on a journey, facing hardships and unexpected discoveries, ultimately learning valuable lessons about friendship, kindness, and pursuing dreams from a mysterious figure they encounter along the way.'

In [ ]:
response = rag_chain.invoke({"input": "Give brief information about Kabir"})
response["answer"]

'Kabir ran a small tea stall in the village, where he made people feel welcome and valued by listening to their stories and sharing laughs. He dreamed of expanding his tea stall into a proper café to create a sense of community.'